# Task
We want to achieve the following workflow:

1. Do structure MD relaxation
2. Heating to targeted temperature (bulk: NPT, particle: NVT)
3. Stablization with NVE. If the sample has a slab geometry, add vaccum in z direction for bulk structure. 
4. Run MD at thermal equilibirium (NVT)

In [1]:
# for deepmd potential use only
element_map={"Mg":0, "Al":1,"Cu":2}
# for EAM potential use only
element_map={'Al':0}

In [2]:
from deepmdem.lammps import MDRelax,MDHeating,MDStabilization,MDEquilibrium
from dflow import Step, Workflow, download_artifact, upload_artifact
from dflow.python import (OP, OPIO, Artifact, OPIOSign, PythonOPTemplate,
                          upload_packages)
from typing import List
from pathlib import Path
from dflow import SlurmRemoteExecutor

def remote_slurm(nodes: int):
    slurm_remote_executor = SlurmRemoteExecutor(
        host="host-address",
        port=22,
        username="username",
        private_key_file="path/to/private_key",
        header=f"#!/bin/bash\n#SBATCH --account=account\n#SBATCH --partition=gpu_partition\n#SBATCH --nodes={nodes}\n#SBATCH --ntasks-per-node=4\n#SBATCH --gres=gpu:4\n#SBATCH --time=2:00:00\n#SBATCH --job-name=deepmd\n#SBATCH -e output.err",
        workdir="~/myscratch/dflow/workflows/{{workflow.name}}/{{pod.name}}",
    )
    return slurm_remote_executor

def md_workflow(temp: float,element_map:dict,running_cores: int,nodes: int, add_vacuum:float, potential: Path, input_structure: Path):
        step0 = Step(
        name="Relax",
        template=PythonOPTemplate(MDRelax, command=["source ~/dflow.env && python"]),
        parameters={
                'pbc':True,
                'element_map':element_map,
                'potential_type':'deepmd',
                "running_cores":running_cores
        },
        artifacts={'atomic_potential':potential,
                'structure':input_structure},
        executor=remote_slurm(nodes)

        )

        step1 = Step(
        name="Heating",
        template=PythonOPTemplate(MDHeating, command=["source ~/dflow.env && python"]),
        parameters={'temp': temp,
                'nsteps': 10000,
                'ensemble':'npt',
                'pres':1,
                'pbc':True,
                'timestep':0.005,
                'neidelay':10,
                'trj_freq':100,
                'element_map':element_map,
                'potential_type':'deepmd',
                "running_cores":running_cores

        },
        artifacts={'atomic_potential':potential,
                'structure':step0.outputs.artifacts["out_structure"]},
        executor=remote_slurm(nodes)
        )

        step2 = Step(
        name="Stabilization",
        template=PythonOPTemplate(MDStabilization, command=["source ~/dflow.env && python"]),
        parameters={'temp': temp,
                'nsteps': 5000,
                'pbc':True,
                'timestep':0.005,
                'neidelay':10,
                'trj_freq':100,
                'pres':1,
                'element_map':element_map,
                'potential_type':'deepmd',
                'add_vacuum':add_vacuum,
                "running_cores":running_cores

        },
        artifacts={'atomic_potential':potential,
                'structure':step1.outputs.artifacts["out_structure"]},
                executor=remote_slurm(nodes)
        )

        step3 = Step(
        name="Equilibrium",
        template=PythonOPTemplate(MDEquilibrium, command=["source ~/dflow.env && python"]),
        parameters={'temp': temp,
                'nsteps': 100000,
                'pbc':True,
                'timestep':0.005,
                'neidelay':10,
                'trj_freq':100,
                'element_map':element_map,
                'potential_type':'deepmd',
                "running_cores":running_cores
                
        },
        artifacts={'atomic_potential':potential,
                'structure':step2.outputs.artifacts["out_structure"]},
                executor=remote_slurm(nodes)
        )
        from dflow import Workflow
        wf = Workflow(name=f"md-{temp}k")
        wf.add(step0)
        wf.add(step1)
        wf.add(step2)
        wf.add(step3)

        wf.submit();

In [3]:
upload_packages.append('../deepmdem/')
potential=upload_artifact('bulk_Al/MgAlCu_compress.pb')
input_structure=upload_artifact('bulk_Al/structure_25c')
element_map={"Mg":0, "Al":1}
running_cores = 8 
nodes = 2
temp_list=[273]
for temp in temp_list:
    md_workflow(temp=temp,element_map=element_map,running_cores=running_cores,nodes=nodes, potential=potential, input_structure=input_structure,add_vacuum=40.0)

Workflow has been submitted (ID: md-273k-wp2jz)
